In [140]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import pmdarima as pm
from sklearn.metrics import r2_score
#from sklearn.metrics import mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression
from matplotlib.pyplot import figure
from statsmodels.tsa.arima.model import ARIMA
import statsmodels as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

In [352]:
class ModelArima:
    def __init__(self):
        pass


    # import data
    def get_data(self):
        df = pd.read_excel('../raw_data/ratios.xlsx')
        return df
  

    def run_model(self, df_init,test_size):
        df = df_init.copy()
        ratio_list = list(df.columns)
        ratio_list.remove('Date')
        predictions = pd.DataFrame()
#         predictions['Date'] = df_init['Date'].iloc[-30:]
        
        
        for i in range(len(ratio_list)):

            #train and test split

            ratio = [ratio_list[i]]

            df = df_init[ratio_list[i]]
            train_size = df.shape[0] - test_size
            index = train_size

            df_train = df.iloc[:index]
            df_test = df.iloc[index:]

            #auto - ARIMA
            model = pm.auto_arima(df,
                            start_p=1, max_p=6,
                            start_q=1, max_q=6,
                            trend='t',
                            max_d = 1,
                            seasonal=False,
                            trace=True,
                                 verbose=0)

            # Build model
            arima = ARIMA(df_train, order=model.get_params().get("order"), trend='t')
            arima = arima.fit()

            # Forecast values
            forecast = arima.forecast(30, alpha=0.05)  # 95% confidence
            predictions[ratio_list[i]] = forecast
        

        predictions.reset_index(inplace=True,drop=True)
        return predictions
        
        
    

In [353]:
model = ModelArima()

In [354]:
df = model.get_data()


In [355]:
model.run_model(df,0)

Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=2.432, Time=0.14 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-0.149, Time=0.10 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=0.474, Time=0.24 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=0.504, Time=0.25 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-0.149, Time=0.08 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 0.834 seconds
Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-2797.848, Time=0.32 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-2793.738, Time=0.10 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-2797.864, Time=0.07 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-2798.757, Time=0.29 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-2793.738, Time=0.11 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-2798.481, Time=0.39 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-2797.255, Time=0.24 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=-2798.757

/home/sam/.pyenv/versions/3.8.12/envs/long-short-trade-ideas-generator/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/home/sam/.pyenv/versions/3.8.12/envs/long-short-trade-ideas-generator/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
/home/sam/.pyenv/versions/3.8.12/envs/long-short-trade-ideas-generator/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/home/sam/.pyenv/versions/3.8.12/envs/long-short-trade-ideas-generator/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1


 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-1863.486, Time=0.24 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-1869.678, Time=0.17 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=-1869.291, Time=0.18 sec


/home/sam/.pyenv/versions/3.8.12/envs/long-short-trade-ideas-generator/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/home/sam/.pyenv/versions/3.8.12/envs/long-short-trade-ideas-generator/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1


 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-1861.487, Time=0.23 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=-1871.008, Time=0.16 sec

Best model:  ARIMA(1,1,1)(0,0,0)[0] intercept
Total fit time: 1.755 seconds
Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-3480.619, Time=0.15 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-3484.055, Time=0.12 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-3482.729, Time=0.11 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-3482.839, Time=0.27 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-3484.055, Time=0.10 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 0.763 seconds
Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-2304.380, Time=0.17 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-2304.343, Time=0.10 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-2306.333, Time=0.24 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-2306.136, Time=0.29 sec
 ARIMA(0,1,0)(0,0,0)[0]

,CMG_ETN,ROST_MMM,RE_ZTS,MCK_GLW,D_WBD,DVA_OMC,CARR_GOOGL,PGR_CTSH,RJF_MSFT,PKG_FIS,Date
0,11.962261,0.773310,1.809165,11.188054,6.627177,1.341770,0.388191,2.011887,0.416957,1.556247,NaT
1,11.959198,0.773647,1.810111,11.200510,6.633858,1.339859,0.388205,2.013297,0.417305,1.557962,NaT
2,11.956135,0.773985,1.811056,11.212966,6.640538,1.338436,0.388218,2.014618,0.417653,1.559677,NaT
3,11.953072,0.774323,1.812002,11.225422,6.647218,1.337292,0.388231,2.015945,0.418002,1.561393,NaT
4,11.950008,0.774661,1.812948,11.237878,6.653899,1.336306,0.388244,2.017272,0.418350,1.563108,NaT
5,11.946945,0.774999,1.813893,11.250334,6.660579,1.335409,0.388258,2.018599,0.418699,1.564823,NaT
6,11.943882,0.775336,1.814839,11.262790,6.667260,1.334564,0.388271,2.019926,0.419047,1.566539,NaT
7,11.940819,0.775674,1.815785,11.275246,6.673940,1.333748,0.388284,2.021253,0.419395,1.568254,NaT
8,11.937756,0.776012,1.816731,11.287702,6.680620,1.332949,0.388298,2.022580,0.419744,1.569969,NaT
9,11.934693,0.776350,1.817676,11.300158,6.687301,1.332159,0.388311,2.023907,0.420092,1.571685,NaT


In [305]:
ratio_list = list(df.columns)


In [308]:
ratio_list.remove('Date')

In [309]:
ratio_list

['CMG_ETN',
 'ROST_MMM',
 'RE_ZTS',
 'MCK_GLW',
 'D_WBD',
 'DVA_OMC',
 'CARR_GOOGL',
 'PGR_CTSH',
 'RJF_MSFT',
 'PKG_FIS']

In [329]:
df_train = df.iloc[:504]
df_train

,Date,CMG_ETN,ROST_MMM,RE_ZTS,MCK_GLW,D_WBD,DVA_OMC,CARR_GOOGL,PGR_CTSH,RJF_MSFT,PKG_FIS
0,2020-09-08,13.506638,0.602499,1.329053,4.895305,3.251052,1.708033,0.378947,1.340740,0.238529,0.687761
1,2020-09-09,13.449365,0.592539,1.296271,4.776018,3.230514,1.795396,0.380226,1.365644,0.229566,0.681509
2,2020-09-10,13.527456,0.584031,1.266311,4.902900,3.140488,1.829054,0.372837,1.378674,0.234225,0.685586
3,2020-09-11,13.214654,0.570319,1.254798,4.882002,3.263059,1.820385,0.376656,1.334546,0.237897,0.700702
4,2020-09-14,12.823698,0.584702,1.257235,4.779497,3.176914,1.764097,0.377221,1.330864,0.243906,0.705164
...,...,...,...,...,...,...,...,...,...,...,...
499,2022-08-31,11.686183,0.691338,1.718840,10.693473,6.127795,1.274888,0.361486,1.941586,0.399166,1.498522
500,2022-09-01,11.624631,0.702062,1.708488,10.741954,6.244125,1.304472,0.362584,1.949874,0.401536,1.505903
501,2022-09-02,11.773634,0.718372,1.745509,10.974962,6.265697,1.325083,0.366991,1.963902,0.404202,1.512838
502,2022-09-06,11.805430,0.743739,1.749554,11.003354,6.452890,1.338287,0.377305,1.968499,0.408292,1.523266


In [351]:
df['Date'].iloc[-30:]

475   2022-07-28
476   2022-07-29
477   2022-08-01
478   2022-08-02
479   2022-08-03
480   2022-08-04
481   2022-08-05
482   2022-08-08
483   2022-08-09
484   2022-08-10
485   2022-08-11
486   2022-08-12
487   2022-08-15
488   2022-08-16
489   2022-08-17
490   2022-08-18
491   2022-08-19
492   2022-08-22
493   2022-08-23
494   2022-08-24
495   2022-08-25
496   2022-08-26
497   2022-08-29
498   2022-08-30
499   2022-08-31
500   2022-09-01
501   2022-09-02
502   2022-09-06
503   2022-09-07
504   2022-09-08
Name: Date, dtype: datetime64[ns]